# Análise de Desempenho em Operações de Substituição de Texto: Threads vs. Execução Sequencial

## Introdução

A computação paralela, particularmente o uso de threads, tornou-se uma ferramenta essencial para otimizar tarefas intensivas em termos de processamento. Em linguagens como Python, onde a natureza interpretada pode apresentar limitações de desempenho, explorar o paralelismo pode oferecer benefícios significativos. No entanto, o ganho de desempenho depende da natureza da tarefa e da eficiência da implementação paralela.

Neste experimento, vamos investigar o impacto do uso de múltiplas threads em uma operação comum: a substituição de texto em arquivos. Com base em textos gerados contendo variáveis em formato de template, avaliaremos as diferenças entre uma implementação sequencial e seu equivalente utilizando paralelismo.

O objetivo é entender como diferentes quantidades de threads influenciam o desempenho e identificar possíveis gargalos.

## Instalação e carregamento de dependências

In [1]:
import tempfile
import os
import random
import string
import glob
from threading import Thread, Semaphore

In [2]:
def populate_file_with_random_content(filepath, file_size):
    random_string = ''.join(random.choices(string.ascii_letters + string.digits, k=file_size))
    with open(filepath, 'w') as f:
        f.write(random_string)

def create_temp_files(directory, num_files, file_size):
    for i in range(num_files):
        filepath = os.path.join(directory, f'temp_file_{i}')
        populate_file_with_random_content(filepath, file_size)

def replace_temp_file(filepath, file_size):
    with open(filepath, 'r') as f:
        _ = f.read()  # Keeping this useless code here to force IO action

    populate_file_with_random_content(filepath, file_size)

def replace_temp_files(directory, file_size):
    for filepath in glob.glob(os.path.join(directory, '*')):
        replace_temp_file(filepath, file_size)

## Definindo runner

In [3]:
semaphore = None

class ReplaceFileThread(Thread):
    __slots__ = ("filepath", "file_size")

    def __init__(self, filepath, file_size):
        super().__init__()
        self.filepath = filepath
        self.file_size = file_size

    def run(self):
        with semaphore:
            replace_temp_file(self.filepath, self.file_size)

In [4]:
def run_experiment(num_threads):
    global semaphore
    semaphore = Semaphore(num_threads)

    with tempfile.TemporaryDirectory() as temp_dir:
        create_temp_files(temp_dir, 256, 4096)

        filepaths = glob.glob(os.path.join(temp_dir, '*'))

        threads = [ReplaceFileThread(filepath, 2048) for filepath in filepaths]

        for thread in threads:
            thread.start()

        for thread in threads:
            thread.join()

In [5]:
%%timeit -r 5 -n 20

run_experiment(1)

233 ms ± 20.3 ms per loop (mean ± std. dev. of 5 runs, 20 loops each)


In [6]:
%%timeit -r 5 -n 20

run_experiment(2)

273 ms ± 20.7 ms per loop (mean ± std. dev. of 5 runs, 20 loops each)


In [7]:
%%timeit -r 5 -n 20

run_experiment(5)

278 ms ± 20.3 ms per loop (mean ± std. dev. of 5 runs, 20 loops each)


In [8]:
%%timeit -r 5 -n 20

run_experiment(10)

223 ms ± 2.58 ms per loop (mean ± std. dev. of 5 runs, 20 loops each)


In [9]:
%%timeit -r 5 -n 20

run_experiment(25)

220 ms ± 2.74 ms per loop (mean ± std. dev. of 5 runs, 20 loops each)


In [10]:
%%timeit -r 5 -n 20

run_experiment(50)

253 ms ± 25.8 ms per loop (mean ± std. dev. of 5 runs, 20 loops each)


In [11]:
%%timeit -r 5 -n 20

run_experiment(100)

261 ms ± 10.9 ms per loop (mean ± std. dev. of 5 runs, 20 loops each)
